In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
train_dataset = datasets.ImageFolder(root = './data/horse-or-human/train',transform = transform)
test_dataset = datasets.ImageFolder(root = './data/horse-or-human/test',transform = transform)
train_loader = DataLoader(train_dataset,batch_size=5,shuffle =True)
test_loader = DataLoader(test_dataset,batch_size=5,shuffle =True)


In [ ]:
img,label = next(iter(train_loader))

torch.Size([5, 3, 32, 32])

In [9]:
train_dataset.classes

['horses', 'humans']

In [ ]:
import torch.nn as nn
from tqdm import tqdm
from torch.optim import Adam
class BasicBlock(nn.Module):
    def __init__(self, int_channels, out_channels, hidden_dim):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(int_channels, hidden_dim, kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(hidden_dim, out_channels,kernel_size=3,padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
    def forward(self, x):
        x = self.relu( self.conv1(x) )
        x = self.relu( self.conv2(x) )
        out = self.pool(x)
        return out

import torch
class CNN(nn.Module):
    def __init__(self, num_class):
        super(CNN,self).__init__()
        self.block1 = BasicBlock(3,32,16)  
        self.block2 = BasicBlock(32,128,64)
        self.block3 = BasicBlock(128,256,128)

        # 분류기
        self.fc1 = nn.Linear( 4096, 2048)
        self.fc2 = nn.Linear(2048 , 256)
        self.fc3 = nn.Linear(256 , num_class)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)  #(N,256,4,4)        
        x = torch.flatten(x, start_dim=1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        out = self.fc3(x)
        return out    

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CNN(2)
model.to(device)

lr = 1e-3
optim = Adam(model.parameters(), lr=lr)
epochs = 20
# 학습루프
for epoch in range(epochs):
    tqdm_obj = tqdm(train_loader,desc=f'epoch : {epoch+1}/{epochs}')
    for data, label in tqdm_obj:
        optim.zero_grad()
        preds = model(data.to(device))
        loss = nn.CrossEntropyLoss()(preds, label.to(device))
        loss.backward()
        optim.step()

        tqdm_obj.set_postfix(loss=loss.item())
        

torch.save(model.state_dict(), 'hm.pth')    

epoch : 10: 100%|██████████| 202/202 [00:39<00:00,  5.17it/s]


epoch : 10  loss : 0.0012821187265217304


epoch : 20: 100%|██████████| 202/202 [00:39<00:00,  5.14it/s]


epoch : 20  loss : 0.0


In [12]:
# 평가
model.load_state_dict(torch.load('hm.pth',map_location=device,weights_only=True))
# 예측
# 평가 루프
# test_loader가 이미 정의되어 있다고 가정
model.eval()  # 평가 모드로 전환 (dropout, batchnorm 등 비활성화)
total_loss = 0.0
total_correct = 0
total_samples = 0

criterion = nn.CrossEntropyLoss()
with torch.no_grad():  # 그래디언트 계산 비활성화
    for data, label in tqdm(test_loader, desc="Evaluating"):
        data, label = data.to(device), label.to(device)
        preds = model(data)
        loss = criterion(preds, label)
        total_loss += loss.item() * data.size(0)  # 배치 손실 합산
        total_correct += (preds.argmax(dim=1) == label).sum().item()
        total_samples += data.size(0)

avg_loss = total_loss / total_samples
accuracy = total_correct / total_samples

print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}")

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: './data/horse-or-human/test\\horse\\horse07-6.png'